# Task 2: Traditional Feature-based Methods

Using effective features over graphs is the key to achieving good test performance. Traditional Machine Learning Pipeline manually designs features for nodes/links/graphs. 

This posts will cover traditional features for node/link/graph level prediction for undirected graphs.


## Node Level Tasks and Features

[Node classification](https://neo4j.com/developer/graph-data-science/node-classification/) is a supervised machine learning (ML) approach whereby existing nodes with known classes can be used to train a model that will learn the classes for nodes where they are unknown.

There are different ways to characterize the structure and position of a node in the network.

**Importance based features** capture the importance of a node in graph. This type of features is useful for predicting influential nodes in a graph.e.g.predicting celebrity users in a social network.
- Node degree
- Different node centrality measures

**Structure based features** : Capture topological properties of local neighborhood around a node.Useful for predicting a particular role a node plays in a graph. e.g. Predicting protein functionality in a
protein protein interaction network.
- Node degree
- Clustering coefficient
- Graphlet count vector


### Node Degree

The degree $k_v$ of node $v$ is the number of edges (neighboring nodes) the node has. Treats all neighboring nodes equally


### Node centrality

Node centrality $C_v$ takes the node importance in a graph into account. There are different ways to model importance.


#### Engienvector centrality

Engienvector centrality models the centrality of node $v$ as the sum of the centrality of neighboring nodes:

$$c_v = \frac{1}{\lambda}\sum_{u \in N(v)}{c_u}$$

Where $\lambda$ is normalization constant. It will turn out to be the largest eigenvalue of A.

Rewrite the recursive equation in the matrix form.

$$\lambda c = A\mathbb{c}$$

Where A is adjacent matrix, $A_uv=1$ if $u \in N(v)$. $c$ is centrality vector and $\lambda$ is eigenvalue.

By Perron-Frobenius Theorem, The largest eigenvalue $\lambda_{max}$ is always positive and unique. And The eigenvector $c_{max}$  corresponding to $\lambda_{max}$ is used for centrality.


#### Betweenness centrality

The intution behind betweenness centrality is that a node is important if it lies on many shortest paths between other nodes. Mathematically, It is defined as follows:

$$c_v = \sum_{s\not =v\not ={t}}\frac{n_1}{n_2}$$

Where $n_1$ is the number of shorted paths between $s$ and $t$ that contains $v$. and $n_2$ is the number of shorted paths between $s$ and $t$.


#### Closeness centrality

The intution behind closeness centrality is that a node is important if it has small shortest path lengths to all other nodes. Mathematically, It is defined as follows:

$$c_v = \frac{1}{\sum_{u\not =v}d_{uv}}$$

Where $d_{uv}$ is the shorest path length betwen $u$ and $v$.

### Clustering Coefficient

Clustering Coefficient measures how connected $v$'s neighboring nodes are. Mathematically, It is defined as follows:

$$e_v = \frac{n}{k_v \choose 2}$$

Where $n$ is the number of edges among neighboring nodes.$k_v \choose 2$ means choose 2 nodes among $k_v$ neighboring nodes for node for node $v$, i.e. it is the number of node pairs among the neighboring nodes for node for node $v$.



### Graphlet Degree Vector

Clustering Coefficient essentially counts the number of triangles in the ego-network. Here ego-network of a given node refers a subgraph induced by the node itself and its neighbors. It is basically its degree-1 neighborhood network. 

We can generalize clustering coefficient by counting the number of pre-specified subgraphs, i.e., graphlets.

For a given node $v$, graphlets are small subgraphs that describe the structure of node $v$'s network neighborhood. 

To formally define graphlets, let's introduce the following 2 concepts:

- ***Induced Subgraph*** is another graph, formed from a subset of vertices and all the edges connecting the vertices in the subsets.

- ***Graph Isomorphism*** two graphs which contains the same number of nodes connected in the same way are said to be isomorphic. 

**Graphlets** are rooted connected induced non-isomorphic subgraphs. 

**Graphlet Degree Vector (GDV)** is a graphlet-based feature which count the number of graphlets for a given node. It provides a measure of **a node's local network topology**. It allows more detailed and comprehensive measure of local topological similarity than node degrees or clustering coefficent. 









## Link Prediction Tasks and Features

Link Prediction is to predict new links based on the existing links. At test time, node pairs (with no existing links) are ranked, and top $K$ node pairs are predicted. The key is to design features for a pair of nodes.

Two formulations of the link prediction task:
- **Links missing at random**: remove a random set of links and then aim to predict them. 
- **Links over time**: given a graph $G[t_{0},t_{0}^{'}]$ defined by edges up to time $t_{0}^{'}$ output a ranked List L of edges that not in $G[t_{0},t_{0}^{'}]$ that are predicted to appear in time $G[t_{1},t_{1}^{'}]$. To evaluate, count how many edges in $L$ are real new edge that appear during the test period.

Link prediction via proximity takes the following steps:
1. For each pair of nodes $(x,y)$ compute score $c(x,y)$ For example, $c(x,y)$ could be the # of common neighbors of $x$ and $y$
2. Sort pairs $(x,y)$ by the decreasing score
$c(x,y)$
3. Predict top $n$ pairs as new links 
4. See which of these links actually appear in $G[t_{1},t_{1}^{'}]$

The question now become what is the formulation of $c$. In other words, how do we compute a score $c(x,y)$ for a given pair of nodes $(x,y)$? $c(x,y)$ is enssentially the link-level feature.


### Distance-based Features
One way is to use shortest-path distance between 2 nodes. However, this does not capture the degree of neighborhood overlap.

### Local Neighborhood Overlap 
The idea here is to capture the number of neighboring nodes shared between two nodes $v_1$ and $v_2$. 
  - **Common Neighbors**
  
  $$N(v_1)\cap N(v_2)$$
  
  - **Jaccard's Coefficient**
  
  $$\frac{N(v_1)\cap N(v_2)}{N(v_1)\cup N(v_2)}$$
  
  - **Adamic-Adar Index**
  
   $$\sum_{u\in N(v_1)\cap N(v_2))}\frac{1}{log(k_u)}$$
  

### Global Neighborhood overlap

The local neighborhood feature is always zero if the two nodes do not have any neighbors in common. However, the two nodes may still potentially be connected in the future.

Global neighborhood overlap metrics resolve the limitation by considering the entire graph. 

**Katz index** counts the number of walks of all lengths between a given pair of nodes $v_1$ and $v_2$. Mathematically it can be defined as follows:

  $$\sum_{l=1}^{\infty}\beta^l \mathbf{\textit{P}}_{v_1v_2}^{(l)}$$
  
Where $\beta^l \in (0,1)$ is a discounting factor. $\mathbf{\textit{P}}_{v_1v_2}^{(l)}$ is the number of walks of lengh $l$ between $v_1$ and $v_2$.  

To compute $\mathbf{\textit{P}}_{v_1v_2}^{(l)}$, we need to make use of the adjacency matrix of the graph $\mathbf{\textit{A}}$. 

- When $l=1$, $\mathbf{\textit{P}}_{v_1v_2}^{(1)}$ specifies the number of walks of length 1 (direct neighborhood) between $v_1$ and $v_2$. $\mathbf{\textit{P}}_{v_1v_2}^{(1)} = \mathbf{\textit{A}}_{v_1v_2}$ 

- When $l=2$, $\mathbf{\textit{P}}_{v_1v_2}^{(2)}$specifies the number of walks of length 2 (neighbor of neighbor) between $v_1$ and $v_2$. It can be derived by summing up the number of works across $v_1$'s neighbors as follows:
$$\mathbf{\textit{P}}_{v_1v_2}^{(2)} = \sum_i A_{v_1i}*\mathbf{\textit{P}}_{v_1v_2}^{(1)} = \mathbf{\textit{A}}_{v_1v_2}^2$$

As such, $\mathbf{\textit{A}}_{v_1v_2}^l $ specifies the number of walks of length $l$ **Katz index** can be write as 

  $$\sum_{l=1}^{\infty}\beta^l \mathbf{\textit{A}}_{v_1v_2}^l$$
  
With this, it can be computed in closed-form


 $$S = \sum_{l=1}^{\infty}\beta^l \mathbf{\textit{A}}^l = (I-\beta\mathbf{\textit{A}})^{-1} -I$$

## Graph-Level Features and Graph Kernels

The graph-level features are designed to characterize the structure of the entire graph.  Kernel methods are widely used for traditional ML for graph level prediction.

In the case of Natural Language Processing, **Bag-of-Words(BoW)** uses word counts as features for documents without consider the ordering of the words.  BOW for a graph then would be to treat nodes as words. However, because it only focus on the node itself by ignoring the links of hte graph, it may lead to same representation for different graph.

To address that, we can use **bag of node degrees**. Node degree combines the information of links and nodes and thus the structure of the graph. As such, it will produce different features for different graphs. 

Both **Graphlet Kernel** and **Weisfeiler-Lehman (WL) Kernel** use Bag-of-* representation of graph, where * is more sophisticated than node degrees.

### Graphlet Kernel


Graph-level graphlet features count the number of different graphlets in a graph. 

It is worth noting that nodes in graphlets here do not need to be connected. In other words, isolated nodes are allowed. And unlike the node-level graphlet, graph-level graphlet are not rooted.

Given graph $G$, and a graphlet list $\mathcal{G}_k=(g_1,g_2,...,g_{n_k})$, define the graphlet count
vector $\mathcal{f}_G \in \mathbb{R}^{n_k}$ as

$$(\mathcal{f}_G)_i = \#(g_i \subseteq G)$$ for $$ i = 1,2,..,n_k$$

Given two graphs, $G$ and $G'$, graphlet kernel is computed as

$$K(G,G')={\mathcal{f}_G}^T\mathcal{f}_{G'} $$

When $G$ and $G'$ have different sizes, it can will greatly skew the value. To address this, we can normalize each feature vector:

$$K(G,G')={\mathcal{h}_G}^T\mathcal{h}_{G'} $$

Where $\mathcal{h}_G = \frac{\mathcal{f}_G}{\sum{\mathcal{f}_G}}$.


### Weisfeiler-Lehman (WL) Kernel

When using graphlet kernel, counting graphlets is expensive. 
- Counting size-$k$ graphlets for a graph with size $n$ by enumeration takes $n^k$. 
- This is unavoidable in the worst-case since subgraph isomorphism test (judging whether a graph is a subgraph of another graph) is NP-hard. 
- If a graph’s node degree is bounded by $n$ , an $O(nd^{k-1})$  algorithm exists to count all the graphlets of size $k$.

Can we design a more efficient graph kernel? 

***Weisfeiler-Lehman (WL) Kernel*** use neighborhood structure to iteratively enrich node vocabulary. It uses a generalized version of bag of node degrees. It can be achieved with the **color refinement algorithm**. Given a graph $G$ with a set of node $V$.
- Assgin an initial color $c^{(0)}(v)$ to each node $v$.
- Iteractively refine node colors by 

    $$c^{(k+1)}(v) = \mathrm{HASH}(\{c^{(k)}(v)\},\{c^{(k)}(u)\}_{u\in N(v)}\})$$

    where $\mathrm{HASH}$ maps different inputs to different colors.

- After $K$ steps of color refinement, $c^{(k)}(v)$ summarizes the structure of $K-hop$ neighborhood.

After color refinement, WL kernel counts number of nodes with a given color. The WL kernel value is computed by the inner product of the color count vectors.

WL kernel is computationally efficient.
- The time complexity for color refinement is $O(K*n_{edge})$ where $n_{edge}$ is the number of edges in the graph.
- The time complexity for kernel value computing is $O(n_{node})$ where $n_{node}$ is the number of nodes in the graph.
- In total, time complexity is linear in the numebr of edges.

## References

\[1\][Featuer Engieering Youtube. Stanford CS224W: Machine Learning with Graphs | 2021](https://www.youtube.com/watch?v=P-m1Qv6-8cI&list=PLoROMvodv4rPLKxIpqhjhPgdQy7imNkDn&index=1)

\[2\][Featuer Engieering Slides. Stanford CS224W: Machine Learning with Graphs | 2021](http://web.stanford.edu/class/cs224w/slides/02-tradition-ml.pdf)

\[3\][传统图机器学习的特征工程-节点【斯坦福CS224W】](https://www.bilibili.com/video/BV1HK411175s/?spm_id_from=333.788&vd_source=fccde7883fbf93ac15b03dee298f9f18)

\[4\][传统图机器学习的特征工程-连接【斯坦福CS224W】](https://www.bilibili.com/video/BV1r3411m7sD/?spm_id_from=333.788)

\[5\][传统图机器学习的特征工程-全图【斯坦福CS224W】](https://www.bilibili.com/video/BV1r3411m7sD/?spm_id_from=333.788)

[6] Shervashidze, Nino, et al. "Efficient graphlet kernels for large graph comparison." Artificial Intelligence and Statistics. 2009.

[7] Shervashidze, Nino, et al. "Weisfeiler-lehman graph kernels." Journal of Machine Learning Research 12.9 (2011).

